In [ ]:
from neo4j import GraphDatabase
from tqdm import tqdm
import config.EnvLoader as el

URI = "neo4j://localhost"
AUTH = ("neo4j", el.NEO4J_PWD)

In [47]:
import json

with open('extracted_chunks_hybrid_multihop_468.json', 'r') as file:
    extracted_chunks_hybrid_tot = json.load(file)

In [2]:
import pickle 

# Open the file in binary mode 
with open('multihop_eval_df.pkl', 'rb') as file: 
	mh_eval = pickle.load(file) 

In [3]:
mh_eval = mh_eval[:100]

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

# Initialize Azure Embeddings
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=el.OPENAI_API_KEY,
    azure_endpoint=el.AZURE_ENDPOINT,
    openai_api_version="2023-03-15-preview",
)

In [5]:
q_diz = {}

for obs in mh_eval:
    q_diz[obs['question'].strip()] = embeddings.embed_query(obs['question'].strip())

# Vector Retrieval on Multihop

In [19]:
extracted_chunks_naive = {}

for obs in mh_eval:
    question = obs['question'].strip()
    answer = obs['answer'].strip()
    diz = {}
    for i in [4,6,8]:
        # Search for top n chunk
        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            retrieved_chunks, _, _ = driver.execute_query(
                """CALL db.index.vector.queryNodes("vector", 100, $embedding)
                    YIELD node, score
                    RETURN elementId(node), node.uri, node.text
                    LIMIT $n_chunks""",
                embedding=q_diz[question], n_chunks = i
            )
        top_chunks_ids = [x["elementId(node)"] for x in retrieved_chunks]
        top_chunks = [x["node.uri"] for x in retrieved_chunks]
        context = [x["node.text"] for x in retrieved_chunks]
        diz[i] = {
            'top_chunks_ids': top_chunks_ids,
            'top_chunks': top_chunks,
            'context': context
        }
    extracted_chunks_naive[question] = diz

In [35]:
naive_precision = {4:[],6:[],8:[]}
naive_recall = {4:[],6:[],8:[]}
naive_detection = {4:[],6:[],8:[]}

n_chunks = [4,6,8]

for obs in mh_eval:
    question = obs["question"].strip()
    answers = obs["chunks_used"]
    for n_chunk in n_chunks:
        found_answers = [x for x in answers if x in extracted_chunks_naive[question][n_chunk]['top_chunks']]
        # Append results for Chunk RAG
        naive_precision[n_chunk].append(len(found_answers)/n_chunk)
        naive_recall[n_chunk].append(len(found_answers)/len(answers))
        naive_detection[n_chunk].append(1 if len(found_answers) == len(answers) else 0)

In [36]:
# Results for 4 retrieved chunks
for i in n_chunks:
    print(f"Results for {i} chunks")
    print(f"Precision: {round(sum(naive_precision[i])/100,3)}")
    print(f"Recall: {round(sum(naive_recall[i])/100,3)}")
    print(f"Detection: {round(sum(naive_detection[i])/100,3)}")

Results for 4 chunks
Precision: 0.175
Recall: 0.357
Detection: 0.09
Results for 6 chunks
Precision: 0.138
Recall: 0.418
Detection: 0.16
Results for 8 chunks
Precision: 0.111
Recall: 0.448
Detection: 0.18


# Hybrid Retrieval on MultiHop

In [ ]:
mh_eval[0]

{'response': 'Question: Which song did Sam Smith contribute to with Band Aid 30 to raise money for the 2014 Ebola crisis?\nAnswer: Sam Smith contributed to the song "Do They Know It\'s Christmas?" with Band Aid 30 to raise money for the 2014 Ebola crisis.\nChunk used: chunk0, chunk2',
 'context': 'INPUT:\n- chunk0: UK Singles Chart records and statistics#Number ones by different artists#1\nCurrently two songs have reached number one four times by different artists: "Unchained Melody" and "Do They Know It\'s Christmas?". Three of the versions of "Unchained Melody" sold over a million copies, while two of the versions of "Do They Know It\'s Christmas?" achieved this. The lyrics of the Band Aid 30 version were changed to give it relevance to the 2014 Ebola crisis. Numerous artists appear on more than one version of "Do They Know It\'s Christmas?".\n- chunk2: Sam Smith (singer)#Music career#2014–2016: In the Lonely Hour and international success#4\nIn June 2014, Smith first appeared on the

In [49]:
hybrid_precision = {4:[],6:[],8:[]}
hybrid_recall = {4:[],6:[],8:[]}
hybrid_detection = {4:[],6:[],8:[]}

n_chunks = [4,6,8]

for obs in mh_eval:
    question = obs["question"].strip()
    answers = obs["chunks_used"]
    # Search answers ID
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        retrieved_chunks, _, _ = driver.execute_query(
            """WITH $uri_ids AS list
            MATCH (a:Chunk)
            WHERE a.uri IN list
            RETURN elementId(a)""",
            uri_ids=answers
        )
    answers_ids = [int(x["elementId(a)"].split(":")[-1]) for x in retrieved_chunks]
    for idx, n_chunk in enumerate(n_chunks):
        found_answers = [x for x in answers_ids if x in extracted_chunks_hybrid_tot[" " + question][idx]]
        # Append results for Chunk RAG
        hybrid_precision[n_chunk].append(len(found_answers)/n_chunk)
        hybrid_recall[n_chunk].append(len(found_answers)/len(answers))
        hybrid_detection[n_chunk].append(1 if len(found_answers) == len(answers) else 0)

In [50]:
for i in n_chunks:
    print(f"Results for {i} chunks")
    print(f"Precision: {round(sum(hybrid_precision[i])/100,3)}")
    print(f"Recall: {round(sum(hybrid_recall[i])/100,3)}")
    print(f"Detection: {round(sum(hybrid_detection[i])/100,3)}")

Results for 4 chunks
Precision: 0.302
Recall: 0.599
Detection: 0.33
Results for 6 chunks
Precision: 0.232
Recall: 0.688
Detection: 0.43
Results for 8 chunks
Precision: 0.18
Recall: 0.711
Detection: 0.48
